<a href="https://colab.research.google.com/github/RashmikaReddy/Named-Entity-Recognition/blob/main/NER_CRF_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from numpy import NaN

In [ ]:
!pip install nltk
import nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
!pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 4.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [ ]:
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite.metrics import flat_precision_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-install-45mgat9o/sklearn-crfsuite_8772983593da44fc959acaeb214de3db
  Running command git clone -q https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-install-45mgat9o/sklearn-crfsuite_8772983593da44fc959acaeb214de3db
     |████████████████████████████████| 1.0 MB 4.5 MB/s 
  Created wheel for sklearn-crfsuite: filename=sklearn_crfsuite-0.3.6-py2.py3-none-any.whl size=10889 sha256=af26860e77de9b736f9ca409c7c29d85f2106f40234c0484ecd6979753aa7b3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-oof8raye/wheels/fb/c8/8a/95b4eccd3a273adbfb0a08d7f0e96d45d9f4aee82015d293c2
Successfully built sklearn-crfsuite


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/EbayChallenge/Train_Tagged_Titles.tsv', sep="\t",on_bad_lines='skip')

In [ ]:
data.head(20)

,Record Number,Title,Token,Tag
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,NaN
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model
5,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Multi-color,Color
6,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,canvas,Fabric Type
7,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Multi-color,Color
8,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,canvas,Fabric Type
9,2,LOUIS VUITTON Petit Noe Drawstring Shoulder Ba...,LOUIS,Brand


In [ ]:
data['Tag'].unique()

array(['Brand', nan, 'MPN', 'Type', 'Model', 'Color', 'Fabric Type',
       'Closure', 'Pattern', 'Material', 'No Tag', 'Product Line',
       'Features', 'Theme', 'Country/Region of Manufacture', 'Occasion',
       'Department', 'Size', 'Handle/Strap Material', 'Handle Style',
       'Accents', 'Style', 'Hardware Material', 'Trim Material',
       'Measurement, dimension', 'Pocket Type', 'Character', 'Obscure',
       'Season', 'Character Family', 'Lining Material', 'Strap Drop',
       'Handle Drop'], dtype=object)

In [ ]:
#Number of texts we have are 4997
data['Record Number'].nunique()

4997

In [ ]:
data_df = data[['Record Number','Token','Tag']]

In [ ]:
data_df = data_df.rename(columns={"Record Number": "sentence_id", "Token": "words","Tag":"labels"})


In [ ]:
data_df

,sentence_id,words,labels
0,1,LOUIS,Brand
1,1,VUITTON,NaN
2,1,M40096,MPN
3,1,Handbag,Type
4,1,Priscilla,Model
...,...,...,...
55075,5000,Top,No Tag
55076,5000,Closure,No Tag
55077,5000,Retail,No Tag
55078,5000,$,No Tag


In [ ]:
data.head(5)

,Record Number,Title,Token,Tag
0,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,LOUIS,Brand
1,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,VUITTON,NaN
2,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,M40096,MPN
3,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Handbag,Type
4,1,LOUIS VUITTON M40096 Handbag Priscilla Multi-c...,Priscilla,Model


In [ ]:
# Using insert(), add empty column at first position
data_df.insert(3,"POS Tagging", " ")
print(data_df)


       sentence_id      words  labels POS Tagging
0                1      LOUIS   Brand            
1                1    VUITTON     NaN            
2                1     M40096     MPN            
3                1    Handbag    Type            
4                1  Priscilla   Model            
...            ...        ...     ...         ...
55075         5000        Top  No Tag            
55076         5000    Closure  No Tag            
55077         5000     Retail  No Tag            
55078         5000          $  No Tag            
55079         5000        595  No Tag            

[55080 rows x 4 columns]


In [ ]:
dictionary = {}
input
unique_tags = set()
prev_rownumber = 1
s = ""
prev_title = ""
for index,row in data.iterrows():
  title = row['Title']
  rownumber = row['Record Number']
  if rownumber != prev_rownumber:
    if len(s) != 0:
      dictionary[prev_title] = s[1:]
      s = ""
      prev_rownumber = rownumber
  if rownumber == prev_rownumber:
    if row['Tag'] == 'No Tag':
      s = s+";"+"O"
      data_df['labels'][index] = 'O'
      unique_tags.add("O")
    elif row['Tag'] is not NaN:
      s = s+";"+"B-"+row['Tag']
      data_df['labels'][index] = "B-"+row['Tag']
      unique_tags.add("B-"+row['Tag'])
      prev_tag = row['Tag']
    elif row['Tag'] is NaN:
      s = s+";"+"I-"+prev_tag
      data_df['labels'][index] = "I-"+prev_tag
      unique_tags.add("I-"+prev_tag)
    prev_rownumber = rownumber
    prev_title = title
dictionary[prev_title] = (s.lstrip()).rstrip()

In [ ]:
dictionary = {}
input
unique_tags = set()
prev_rownumber = 1
s = ""
prev_title = ""
for index,row in data.iterrows():
  title = row['Title']
  rownumber = row['Record Number']
  if rownumber != prev_rownumber:
    if len(s) != 0:
      dictionary[prev_title] = s[1:]
      s = ""
      prev_rownumber = rownumber
  if rownumber == prev_rownumber:
    if row['Tag'] == 'No Tag':
      s = s+";"+"O"
      data_df['labels'][index] = 'O'
      unique_tags.add("O")
    elif row['Tag'] is not NaN:
      s = s+";"+"B-"+row['Tag']
      data_df['labels'][index] = "B-"+row['Tag']
      unique_tags.add("B-"+row['Tag'])
      prev_tag = row['Tag']
    elif row['Tag'] is NaN:
      s = s+";"+"I-"+prev_tag
      data_df['labels'][index] = "I-"+prev_tag
      unique_tags.add("I-"+prev_tag)
    prev_rownumber = rownumber
    prev_title = title
dictionary[prev_title] = (s.lstrip()).rstrip()

<ipython-input-17-6a8626bb8c92>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['labels'][index] = "B-"+row['Tag']
<ipython-input-17-6a8626bb8c92>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['labels'][index] = "I-"+prev_tag
<ipython-input-17-6a8626bb8c92>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['labels'][index] = 'O'


In [ ]:
print(data_df)


       sentence_id      words   labels POS Tagging
0                1      LOUIS  B-Brand            
1                1    VUITTON  I-Brand            
2                1     M40096    B-MPN            
3                1    Handbag   B-Type            
4                1  Priscilla  B-Model            
...            ...        ...      ...         ...
55075         5000        Top        O            
55076         5000    Closure        O            
55077         5000     Retail        O            
55078         5000          $        O            
55079         5000        595        O            

[55080 rows x 4 columns]


In [ ]:
for index,row in data_df.iterrows():
  word = [row['words']]
  data_df['POS Tagging'][index] = nltk.tag.pos_tag(word)[0][1]


<ipython-input-19-910f905e52f1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['POS Tagging'][index] = nltk.tag.pos_tag(word)[0][1]


In [ ]:
print(data_df)


       sentence_id      words   labels POS Tagging
0                1      LOUIS  B-Brand         NNP
1                1    VUITTON  I-Brand          NN
2                1     M40096    B-MPN          NN
3                1    Handbag   B-Type          NN
4                1  Priscilla  B-Model          NN
...            ...        ...      ...         ...
55075         5000        Top        O          JJ
55076         5000    Closure        O          NN
55077         5000     Retail        O          JJ
55078         5000          $        O           $
55079         5000        595        O          CD

[55080 rows x 4 columns]


In [ ]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['words'].values.tolist(),
                                                       s['POS Tagging'].values.tolist(),
                                                       s['labels'].values.tolist())]
        self.grouped = self.df.groupby("sentence_id").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [ ]:
#Displaying one full sentence
getter = sentence(data_df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'LOUIS VUITTON M40096 Handbag Priscilla Multi-color canvas Multi-color canvas'

In [ ]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

None


In [ ]:
sentences = getter.sentences


In [ ]:
print(sentences)

[[('LOUIS', 'NNP', 'B-Brand'), ('VUITTON', 'NN', 'I-Brand'), ('M40096', 'NN', 'B-MPN'), ('Handbag', 'NN', 'B-Type'), ('Priscilla', 'NN', 'B-Model'), ('Multi-color', 'NN', 'B-Color'), ('canvas', 'NN', 'B-Fabric Type'), ('Multi-color', 'NN', 'B-Color'), ('canvas', 'NN', 'B-Fabric Type')], [('LOUIS', 'NNP', 'B-Brand'), ('VUITTON', 'NN', 'I-Brand'), ('Petit', 'NN', 'B-Model'), ('Noe', 'NN', 'I-Model'), ('Drawstring', 'VBG', 'B-Closure'), ('Shoulder', 'NN', 'B-Type'), ('Bag', 'NN', 'B-Type'), ('Monogram', 'NN', 'B-Pattern'), ('Leather', 'NN', 'B-Material'), ('M42226', 'NN', 'B-MPN'), ('39SD442', 'CD', 'O')], [('LOUIS', 'NNP', 'B-Brand'), ('VUITTON', 'NN', 'I-Brand'), ('Damier', 'NN', 'B-Product Line'), ('Azur', 'NN', 'B-Color'), ('Pochette', 'NN', 'B-Product Line'), ('Bosphore', 'IN', 'B-Model'), ('Shoulder', 'NN', 'B-Type'), ('Bag', 'NN', 'B-Type'), ('N51112', 'NN', 'B-MPN'), ('LV', 'NN', 'B-Brand'), ('Auth', 'NN', 'O'), ('yt523', 'NN', 'O')], [('GUCCI', 'NN', 'B-Brand'), ('Bamboo', 'NN', 

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
      
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:
print(X[0:5])

[[{'bias': 1.0, 'word.lower()': 'louis', 'word[-3:]': 'UIS', 'word[-2:]': 'IS', 'word.isupper()': True, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NNP', 'postag[:2]': 'NN', 'BOS': True, '+1:word.lower()': 'vuitton', '+1:word.istitle()': False, '+1:word.isupper()': True, '+1:postag': 'NN', '+1:postag[:2]': 'NN'}, {'bias': 1.0, 'word.lower()': 'vuitton', 'word[-3:]': 'TON', 'word[-2:]': 'ON', 'word.isupper()': True, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'NN', 'postag[:2]': 'NN', '-1:word.lower()': 'louis', '-1:word.istitle()': False, '-1:word.isupper()': True, '-1:postag': 'NNP', '-1:postag[:2]': 'NN', '+1:word.lower()': 'm40096', '+1:word.istitle()': True, '+1:word.isupper()': True, '+1:postag': 'NN', '+1:postag[:2]': 'NN'}, {'bias': 1.0, 'word.lower()': 'm40096', 'word[-3:]': '096', 'word[-2:]': '96', 'word.isupper()': True, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'NN', 'postag[:2]': 'NN', '-1:word.lower()': 'vuitton', '-1:word

In [ ]:
print(y[0:5])

[['B-Brand', 'I-Brand', 'B-MPN', 'B-Type', 'B-Model', 'B-Color', 'B-Fabric Type', 'B-Color', 'B-Fabric Type'], ['B-Brand', 'I-Brand', 'B-Model', 'I-Model', 'B-Closure', 'B-Type', 'B-Type', 'B-Pattern', 'B-Material', 'B-MPN', 'O'], ['B-Brand', 'I-Brand', 'B-Product Line', 'B-Color', 'B-Product Line', 'B-Model', 'B-Type', 'B-Type', 'B-MPN', 'B-Brand', 'O', 'O'], ['B-Brand', 'B-Product Line', 'B-Features', 'B-Type', 'B-Type', 'B-Material', 'B-Color', 'O', 'B-MPN'], ['O', 'O', 'O', 'B-Brand', 'B-Product Line', 'I-Product Line', 'B-Material', 'I-Material', 'B-Type', 'B-Type', 'B-Color', 'O', 'O', 'O']]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [ ]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)


In [ ]:
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

In [ ]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

**Evaluating the model performance.**

We will use precision, recall and f1-score metrics to evaluate the performance of the model since the accuracy is not a good metric for this dataset because we have an unequal number of data points in each class.

In [ ]:
f1_score = flat_f1_score(y_test, y_pred, average = 'macro')
print(f1_score)

0.5891590907214499


In [ ]:
print(y_test)

[['B-Brand', 'I-Brand', 'B-Model', 'B-Pattern', 'B-Material', 'B-Features', 'B-Size', 'O', 'B-Type', 'B-Type', 'B-Type'], ['O', 'B-Brand', 'I-Brand', 'B-Pattern', 'B-Model', 'I-Model', 'B-Type', 'B-Type', 'B-MPN', 'O', 'B-Brand', 'O'], ['O', 'B-Brand', 'B-Model', 'O', 'O', 'B-Pattern', 'B-Color', 'O', 'B-Color', 'B-Material'], ['O', 'B-Brand', 'I-Brand', 'B-Type'], ['O', 'O', 'O', 'O', 'B-Brand', 'I-Brand', 'B-Color', 'B-Pattern', 'B-Type'], ['B-Brand', 'B-Accents', 'B-Model', 'B-Type', 'B-Type', 'B-Material', 'B-Color', 'O', 'O', 'B-Country/Region of Manufacture', 'B-MPN'], ['B-Brand', 'B-Product Line', 'B-Color', 'O', 'B-Color', 'B-Material', 'B-Closure', 'B-Type', 'O', 'O', 'B-Type', 'O', 'O'], ['B-Brand', 'B-Size', 'B-Type', 'B-Material', 'B-Type', 'I-Type', 'B-Accents', 'B-MPN'], ['B-Brand', 'I-Brand', 'B-Model', 'B-Accents', 'B-Material', 'B-Type', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-Brand', 'B-Type', 'B-Type', 'I-Type', 'B-Type', 'B-Brand', 'B-Fabric Type', 'B-Material', 'B-Color

In [ ]:
print(y_pred)

[['B-Brand', 'I-Brand', 'B-Model', 'B-Pattern', 'B-Material', 'B-Features', 'B-Size', 'O', 'B-Type', 'B-Type', 'B-Type'], ['O', 'B-Brand', 'I-Brand', 'B-Pattern', 'B-Model', 'I-Model', 'B-Type', 'B-Type', 'B-MPN', 'O', 'B-Brand', 'O'], ['O', 'B-Brand', 'B-Model', 'I-Model', 'O', 'B-Pattern', 'B-Color', 'O', 'B-Color', 'B-Material'], ['O', 'B-Brand', 'I-Brand', 'B-Type'], ['O', 'O', 'O', 'O', 'B-Brand', 'I-Brand', 'B-Color', 'B-Pattern', 'B-Type'], ['B-Brand', 'B-Accents', 'B-Model', 'B-Type', 'B-Type', 'B-Material', 'B-Color', 'O', 'O', 'O', 'O'], ['B-Brand', 'B-Model', 'B-Color', 'O', 'B-Color', 'B-Material', 'B-Closure', 'B-Type', 'O', 'O', 'B-Type', 'O', 'O'], ['B-Brand', 'B-Size', 'B-Type', 'B-Material', 'B-Type', 'I-Type', 'B-Accents', 'O'], ['B-Brand', 'I-Brand', 'B-Model', 'B-Accents', 'B-Material', 'B-Type', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-Brand', 'B-Type', 'B-Type', 'I-Type', 'B-Type', 'B-Brand', 'B-Fabric Type', 'B-Material', 'B-Color', 'B-Color', 'O'], ['O', 'B-Brand', 'B

In [ ]:
report = flat_classification_report(y_test,y_pred)
print(report)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  labels : list, optional
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  labels : list, optional


                                 precision    recall  f1-score   support

                      B-Accents       0.79      0.71      0.75       190
                        B-Brand       0.94      0.94      0.94       914
                    B-Character       1.00      0.18      0.31        11
             B-Character Family       1.00      0.20      0.33         5
                      B-Closure       0.87      0.89      0.88        61
                        B-Color       0.92      0.93      0.93       876
B-Country/Region of Manufacture       0.83      0.74      0.79        47
                   B-Department       0.96      0.94      0.95       176
                  B-Fabric Type       0.95      0.92      0.93       122
                     B-Features       0.79      0.68      0.73       119
                  B-Handle Drop       0.00      0.00      0.00         3
                 B-Handle Style       0.88      0.86      0.87        70
        B-Handle/Strap Material       0.95      0.

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  labels : list, optional


Defining the parameters for LSTM network

In [ ]:
# Number of data points passed in each iteration
batch_size = 64 
# Passes through entire dataset
epochs = 8
# Maximum length of review
max_len = 75 
# Dimension of embedding vector
embedding = 40 